## **Notebook com os Modelos finais**

## **Introdução**
Este notebook tem por objetivo implementar e comparar diferentes algoritmos de aprendizado de máquina supervisionado de classificação. A partir de características como composição, tamanho e tempo de exposição de diferentes materiais, será prevista sua toxicidade.

Os algoritmos utilizados serão os de k vizinhos mais próximos (knn), árvore de decisão, floresta aleatória, Naive Bayes categórico, Support Vector Classifier (SVC) e Stacking. O desempenho de cada um deles será estimado por validação cruzada do tipo k-fold e os hiperparâmetros dos modelos serão otimizados com o ``optuna``. Além disso, um modelo baseline será estabelecido para fins de comparação.

### **Importando as bibliotecas necessárias**

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from optuna import create_study
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MaxAbsScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree

### **Importando o dataset**

In [2]:
df = pd.read_csv("datasets/HA3B.csv")

display(df.shape)
display(df.head())

(666, 17)

,Material_type,Core_size,Hydro_size,Surface_charge,Surface_area,Formation_enthalpy,Conduction_band,Valence_band,Electronegativity,Assay,Cell_name,Cell_species,Cell_origin,Cell_type,Exposure_time,Exposure_dose,Toxicity
0,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,MTT,HCMEC,Human,Blood,Normal,24,0.001,Nontoxic
1,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,MTT,HCMEC,Human,Blood,Normal,24,0.010,Nontoxic
2,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,MTT,HCMEC,Human,Blood,Normal,24,0.100,Nontoxic
3,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,MTT,HCMEC,Human,Blood,Normal,24,1.000,Nontoxic
4,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,MTT,HCMEC,Human,Blood,Normal,24,5.000,Nontoxic


### **Definindo as ``FEATURES`` e o ``TARGET``**

In [3]:
FEATURES_NUM = ["Core_size", 
                "Hydro_size", 
                "Surface_charge", 
                "Surface_area", 
                "Formation_enthalpy", 
                "Conduction_band", 
                "Valence_band", 
                "Electronegativity", 
                "Exposure_time", 
                "Exposure_dose"
]

FEATURES_DUMMY = ["Material_type", "Assay", "Cell_name", "Cell_species", "Cell_origin", "Cell_type"]

TARGET = ["Toxicity"]

### **Evitando vazamento de dados pelo ``groupby()``**
Pode ser que vários dados do dataset sejam iguais em todos os atributos, diferindo (ou não) apenas no target. Dessa forma, alguns desses dados poderiam acabar sendo usado na etapa de treino e outros na fase de teste. Isso faria com que a métrica não refletisse o real desempenho do modelo; afinal, ele já "conheceria" alguns dados de teste, o que acarreta uma previsão enviesada.

Para evitar esse tipo de vazamento, *antes* do split de treino e teste, é necessário acabar com essa redundância. Isso é feito agrupando todos os dados duplicados em um só: os atributos continuam os mesmos, mas apenas um valor de target é utilizado, a partir de uma estatística dos dados originais. Como o target é categórico, será usada a moda.

Importante que os dados duplicados não precisam ser cópias idênticas: se todos os atributos forem muito próximos (apesar de não serem iguais), o vazamento de dados ocorrerá da mesma maneira. Por isso, antes de identificar dados repetidos e agrupá-los, arredonda-se o valor de cada atributo em uma casa adequada.

#### **Arredondando**

In [4]:
casa_arredondamento = {
    "Core_size": 0, 
    "Hydro_size": 0, 
    "Surface_charge": 0, 
    "Surface_area": 0, 
    "Formation_enthalpy": 0, 
    "Conduction_band": 0,
    "Valence_band": 0, 
    "Electronegativity": 0, 
    "Exposure_time": 1, 
    "Exposure_dose": 1,
}

df_round = df.round(casa_arredondamento)

df_round.shape

(666, 17)

Como nós podemos ver pelo código abaixo esse dataset possui alguns dados duplicados

In [5]:
num_duplicados = df_round[FEATURES_NUM + FEATURES_DUMMY].duplicated().sum()
print(f"Número de linhas duplicadas em df_round: {num_duplicados}")

Número de linhas duplicadas em df_round: 43


#### **Agrupando**

O [``agg()``](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html) é um método do pandas usado para aplicar uma ou mais operações de agregação em grupos de dados.

In [6]:
from scipy import stats

def calcular_moda(serie):
    """Calcula a moda de uma série, retornando o primeiro valor se não houver moda clara"""
    moda = stats.mode(serie)
    return moda[0]

calcular_moda([1,1,1,1,9])

np.int64(1)

In [7]:
from scipy import stats

df_grouped = df_round.groupby(FEATURES_NUM + FEATURES_DUMMY, sort=False)

def calcular_moda(serie):
    """Calcula a moda de uma série"""
    moda = serie.mode()
    return moda[0]

agg_dict = {
    **{col: "mean" for col in FEATURES_NUM},
    **{col: calcular_moda for col in FEATURES_DUMMY},
    "Toxicity": calcular_moda
}

df_grouped = df_grouped.agg(agg_dict)
df_grouped = df_grouped.reset_index(drop=True)

print(f"O shape (linhas x colunas) do df_round é: {df_round.shape}")
print(f"O shape (linhas x colunas) do df_tratado é: {df_grouped.shape}")

O shape (linhas x colunas) do df_round é: (666, 17)
O shape (linhas x colunas) do df_tratado é: (623, 17)


In [8]:
num_duplicados = df_grouped[FEATURES_NUM + FEATURES_DUMMY].duplicated().sum()
print(f"Número de linhas duplicadas em df_grouped: {num_duplicados}")

Número de linhas duplicadas em df_grouped: 0


Como podemos ver não há mais valores duplicados

### **Fazendo a codificação One-Hot**
Os algoritmos de aprendizado de máquina utilizados exigem que todos os atributos sejam numéricos. Dessa forma, é necessário transformar os dados qualitativos adequadamente.  O codificador One-Hot transforma uma coluna de dados categóricos em várias colunas, cada qual representando um dos rótulos possíveis; se o dado originalmente tinha aquele rótulo, atribui-se o valor 1, caso contrário preenche-se com 0.

In [9]:
encoder = OneHotEncoder(sparse_output=False, dtype=np.int32)
dummy_encoded = encoder.fit_transform(df_grouped[FEATURES_DUMMY])

dummy_columns = encoder.get_feature_names_out(FEATURES_DUMMY)
df_dummy = pd.DataFrame(dummy_encoded, columns=dummy_columns, index=df_grouped.index)

df_dummy = pd.concat([df_grouped[FEATURES_NUM + TARGET], df_dummy], axis=1)
FEATURES_FINAL = FEATURES_NUM + list(dummy_columns)

display(df_dummy.shape)
display(df_dummy.head(5))

(623, 63)

,Core_size,Hydro_size,Surface_charge,Surface_area,Formation_enthalpy,Conduction_band,Valence_band,Electronegativity,Exposure_time,Exposure_dose,...,Cell_species_Mouse,Cell_origin_Blood,Cell_origin_Breast,Cell_origin_Liver,Cell_origin_Lung,Cell_origin_Mesothelium,Cell_origin_Nose,Cell_origin_Skin,Cell_type_Cancer,Cell_type_Normal
0,40.0,267.0,36.0,65.0,-17.0,-2.0,-10.0,6.0,24.0,0.0,...,0,1,0,0,0,0,0,0,0,1
1,40.0,267.0,36.0,65.0,-17.0,-2.0,-10.0,6.0,24.0,0.1,...,0,1,0,0,0,0,0,0,0,1
2,40.0,267.0,36.0,65.0,-17.0,-2.0,-10.0,6.0,24.0,1.0,...,0,1,0,0,0,0,0,0,0,1
3,40.0,267.0,36.0,65.0,-17.0,-2.0,-10.0,6.0,24.0,5.0,...,0,1,0,0,0,0,0,0,0,1
4,40.0,267.0,36.0,65.0,-17.0,-2.0,-10.0,6.0,24.0,10.0,...,0,1,0,0,0,0,0,0,0,1


### **Definindo os dados de treino e de teste**
Com o dataframe devidamente tratado, pode ser feita a divisão dos dados.

In [10]:
TAMANHO_TESTE = 0.25
SEED = 404

valores_target = df_dummy[TARGET].values.ravel()

df_treino, df_teste = train_test_split(df_dummy, test_size=TAMANHO_TESTE, random_state=SEED, stratify=valores_target)

X_teste = df_teste.reindex(FEATURES_FINAL, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()

X_treino = df_treino.reindex(FEATURES_FINAL, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()

### **Criando os modelos e espaços de busca**

Serão criados os seguintes modelos para comparação:

- Baseline (dummyclassifier)
- KNN classifier
- Decison Tree
- Random Florest
- SVC
- Naive Bayes
- Stacking?


#### **Baseline**

O baseline estabelece uma referência mínima de desempenho, onde qualquer modelo útil deve superar essa referência.

No caso, foi utilizado o ``DummyClassifier(strategy='most_frequent')``, que prevê sempre a moda dos valores de y.

In [11]:
modelo_baseline = DummyClassifier(strategy="most_frequent")

modelo_baseline.fit(X_treino, y_treino)

y_pred_baseline = modelo_baseline.predict(X_teste)

accuracy_baseline = accuracy_score(y_teste, y_pred_baseline)
accuracy_baseline

0.717948717948718

#### **Implementação dos Modelos com o Optuna**
O Optuna é um framework de otimização de hiperparâmetros. Ele automatiza o processo de enontrar o conjunto ótimo de hiperparâmetros para um dado modelo, almeando minimizar ou maximizar uma função objetiva específica.

No Optuna, os ``Trials`` são as tentativas com diferentes combinações de hiperparâmetros e o ``Study`` é o conjunto de trials para um determinado objetivo.

Nesse contexto, a função ``cria_instancia_modelo`` serve para criar uma instância do modelo escolhido, recebendo um trial.
Para definir o espaço do dicionário dos parâmetros é usado o ``trial.suggest_*()``.

Além disso, foi utilizada a decisão de normalização dos dados como um hiperparâmetro adicional. Para isso foi criado um curto **pipeline** com o ``make_pipeline()``

#### **Instância K-NN**

[explicar sobre o modelo knn]

In [12]:
def cria_instancia_knn(trial):
    """Cria uma instância de um modelo KNN."""

    parametros = {
        "n_neighbors": trial.suggest_int("num_vizinhos", 1, 200),
        "weights": trial.suggest_categorical("pesos", ["uniform", "distance"]),
        "p": trial.suggest_int("tipo_distancia", 1, 2),
        "n_jobs": -1,
    }

    normalizar = trial.suggest_categorical("normalizar", [True, False])

    if normalizar:
        tipo_normaliacao = trial.suggest_categorical("tipo_norm", ["Standard", "MinMax", "MaxAbs"])

        if tipo_normaliacao == "Standard":
            normalizador = StandardScaler()
        elif tipo_normaliacao == "MinMax":
            normalizador = MinMaxScaler()
        elif tipo_normaliacao == "MaxAbs":
            normalizador = MaxAbsScaler()
            
        modelo_knn = make_pipeline(
            normalizador,
            KNeighborsClassifier(**parametros)
        )
    
    else:
        modelo_knn = KNeighborsClassifier(**parametros)


    return modelo_knn

#### **Instância Árvore de Decisão**

[explicar sobre o modelo de árvore de decisão]

In [13]:
def cria_instancia_dtree(trial):
    """Cria a instância de um modelo de árvore de decisão"""

    parametros = {
        "max_depth": trial.suggest_int("profundidade", 2, 600),
        "criterion": trial.suggest_categorical("critério", ['entropy', 'log_loss', 'gini']),
        "min_samples_split": trial.suggest_int("min_exemplos_split", 2, 200),
        "min_samples_leaf": trial.suggest_int("min_exemplos_folha", 1, 100),
        "max_features": trial.suggest_float("num_max_features", 0, 1),
        "random_state": SEED,
    }

    normalizar = trial.suggest_categorical("normalizar", [True, False])

    normalizar = trial.suggest_categorical("normalizar", [True, False])

    if normalizar:
        tipo_normaliacao = trial.suggest_categorical("tipo_norm", ["Standard", "MinMax", "MaxAbs"])

        if tipo_normaliacao == "Standard":
            normalizador = StandardScaler()
        elif tipo_normaliacao == "MinMax":
            normalizador = MinMaxScaler()
        elif tipo_normaliacao == "MaxAbs":
            normalizador = MaxAbsScaler()
            
        modelo_dtree = make_pipeline(
            normalizador,
            DecisionTreeClassifier(**parametros)
        )
    
    else:
        modelo_dtree = DecisionTreeClassifier(**parametros)

    return modelo_dtree

#### **Instância Naive Bayes**

[explicar sobre o modelo naive bayes]

In [ ]:
def cria_instancia_nbayes(trial):
    """Cria a instância de um modelo de árvore de decisão"""

    parametros = {
        "alpha": trial.suggest_float("alfa", 0, 1),
        "fit_prior": trial.suggest_categorical("prioridade", [True, False]),
    }

    normalizar = trial.suggest_categorical("normalizar", [True, False])

    if normalizar:
        tipo_normaliacao = trial.suggest_categorical("tipo_norm", ["Standard", "MinMax", "MaxAbs"])

        if tipo_normaliacao == "Standard":
            normalizador = StandardScaler()
        elif tipo_normaliacao == "MinMax":
            normalizador = MinMaxScaler()
        elif tipo_normaliacao == "MaxAbs":
            normalizador = MaxAbsScaler()
            
        modelo_nbayes = make_pipeline(
            normalizador,
            CategoricalNB(**parametros)
        )
    
    else:
        modelo_nbayes = CategoricalNB(**parametros)

    return modelo_nbayes

### **Função Objetivo**

A seguir foi criada a função objetivo que é a função que irá computar a métrica de interesse. Neste caso, a métrica de interesse é a acurácia, obtida por validação cruzada.

In [ ]:
def funcao_objetivo(trial, X, y, num_folds, modelo="knn"):
    """Função objetivo do optuna"""

    if modelo == "knn":
        modelo = cria_instancia_knn(trial)
    elif modelo == "dtree":
        modelo = cria_instancia_dtree(trial)
    elif modelo == "nbayes":
        modelo = cria_instancia_nbayes(trial)

    metricas = cross_val_score(
        modelo, 
        X, 
        y, 
        scoring="accuracy",
        cv=num_folds,
        )
    
    return metricas.mean()


### **Otimizando os Hiperparâmetros**

A seguir foram criados os estudos (conjunto de trials) usando o ``create_study()``, cujo o argumento ``direction='maximizar'`` tem como objetivo minimizar a [métrica]. Foi o utilizado o ``storage`` para armazenar o progreso da busca e o  ``load_if_exists`` para que seja possível continuar a busca de onde ela parou.

In [ ]:
NOME_DO_ESTUDO_KNN = "knn_nanotoxiclogia_optuna"
NOME_DO_ESTUDO_DTREE = "dtree_nanotoxiclogia_optuna"
NOME_DO_ESTUDO_NBAYES = "nbayes_nanotoxiclogia_optuna"

objeto_de_estudo_knn = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO_KNN,
    storage=f"sqlite:///{NOME_DO_ESTUDO_KNN}.db",
    load_if_exists=True,
)

objeto_de_estudo_dtree = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO_DTREE,
    storage=f"sqlite:///{NOME_DO_ESTUDO_DTREE}.db",
    load_if_exists=True,
)

objeto_de_estudo_nbayes = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO_NBAYES,
    storage=f"sqlite:///{NOME_DO_ESTUDO_NBAYES}.db",
    load_if_exists=True,
)

[I 2025-10-27 22:45:52,023] Using an existing study with name 'knn_nanotoxiclogia_optuna' instead of creating a new one.
[I 2025-10-27 22:45:52,580] A new study created in RDB with name: dtree_nanotoxiclogia_optuna


Para realmente rodar o otimizador precisamos de uma função objetivo que tenha apenas um argumento, o `trial`. Para isso vamos definir a `funcao_objetivo_parcial`.

Serão usados ``modelo='modelo'`` para cada modelo e um número de fols na validação cruzada igual a 10.

In [ ]:
NUM_FOLDS = 10

def funcao_objetivo_parcial_knn(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS, modelo="knn")

def funcao_objetivo_parcial_dtree(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS, modelo="dtree")

def funcao_objetivo_parcial_nbayes(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS, modelo="nbayes")

Agora podemos definiro o número de novos trials e rodar cada otimização de modelo separadamente.

In [ ]:
NUM_TENTATIVAS = 3
objeto_de_estudo_knn.optimize(funcao_objetivo_parcial_knn, n_trials=NUM_TENTATIVAS)

In [ ]:
NUM_TENTATIVAS = 3
objeto_de_estudo_dtree.optimize(funcao_objetivo_parcial_dtree, n_trials=NUM_TENTATIVAS)

[I 2025-10-27 22:46:21,073] Trial 20 finished with value: 0.7259944495837187 and parameters: {'profundidade': 224, 'critério': 'gini', 'min_exemplos_split': 198, 'min_exemplos_folha': 89, 'num_max_features': 0.7915300025730188, 'normalizar': True, 'tipo_norm': 'MaxAbs'}. Best is trial 6 with value: 0.8413043478260871.
[I 2025-10-27 22:46:21,310] Trial 21 finished with value: 0.719472710453284 and parameters: {'profundidade': 464, 'critério': 'entropy', 'min_exemplos_split': 81, 'min_exemplos_folha': 71, 'num_max_features': 0.40196718631749484, 'normalizar': True, 'tipo_norm': 'MaxAbs'}. Best is trial 6 with value: 0.8413043478260871.
[I 2025-10-27 22:46:21,552] Trial 22 finished with value: 0.7644773358001851 and parameters: {'profundidade': 558, 'critério': 'entropy', 'min_exemplos_split': 89, 'min_exemplos_folha': 61, 'num_max_features': 0.3578434593203361, 'normalizar': True, 'tipo_norm': 'MaxAbs'}. Best is trial 6 with value: 0.8413043478260871.
[I 2025-10-27 22:46:21,791] Trial 23

In [ ]:
NUM_TENTATIVAS = 3
objeto_de_estudo_nbayes.optimize(funcao_objetivo_parcial_nbayes, n_trials=NUM_TENTATIVAS)

[I 2025-10-27 22:46:21,073] Trial 20 finished with value: 0.7259944495837187 and parameters: {'profundidade': 224, 'critério': 'gini', 'min_exemplos_split': 198, 'min_exemplos_folha': 89, 'num_max_features': 0.7915300025730188, 'normalizar': True, 'tipo_norm': 'MaxAbs'}. Best is trial 6 with value: 0.8413043478260871.
[I 2025-10-27 22:46:21,310] Trial 21 finished with value: 0.719472710453284 and parameters: {'profundidade': 464, 'critério': 'entropy', 'min_exemplos_split': 81, 'min_exemplos_folha': 71, 'num_max_features': 0.40196718631749484, 'normalizar': True, 'tipo_norm': 'MaxAbs'}. Best is trial 6 with value: 0.8413043478260871.
[I 2025-10-27 22:46:21,552] Trial 22 finished with value: 0.7644773358001851 and parameters: {'profundidade': 558, 'critério': 'entropy', 'min_exemplos_split': 89, 'min_exemplos_folha': 61, 'num_max_features': 0.3578434593203361, 'normalizar': True, 'tipo_norm': 'MaxAbs'}. Best is trial 6 with value: 0.8413043478260871.
[I 2025-10-27 22:46:21,791] Trial 23

### **Vizualizando os Resultados**

Agora podemos analizar o foi obtido com cada modelo e a partir disso definir qual deles teve o melhor resultado.

#### **Resultado K-NN:**

In [19]:
melhor_trial_knn = objeto_de_estudo_knn.best_trial

print(f"Número do melhor trial K-NN: {melhor_trial_knn.number}")
print(f"Parâmetros do melhor trial : {melhor_trial_knn.params}")

Número do melhor trial K-NN: 17
Parâmetros do melhor trial : {'num_vizinhos': 2, 'pesos': 'distance', 'tipo_distancia': 1, 'normalizar': False}


#### **Resultado Árvore de Decisão:**

In [21]:
melhor_trial_dtree = objeto_de_estudo_dtree.best_trial

print(f"Número do melhor trial da Árvore de Decisão: {melhor_trial_dtree.number}")
print(f"Parâmetros do melhor trial da Árvore de Decisão: {melhor_trial_dtree.params}")

Número do melhor trial da Árvore de Decisão: 6
Parâmetros do melhor trial da Árvore de Decisão: {'profundidade': 91, 'critério': 'gini', 'min_exemplos_split': 110, 'min_exemplos_folha': 76, 'num_max_features': 0.9539556990162101, 'normalizar': True, 'tipo_norm': 'MaxAbs'}


#### **Resultado Naive Bayes:**

In [ ]:
melhor_trial_nbayes = objeto_de_estudo_nbayes.best_trial

print(f"Número do melhor trial do Naive Bayes: {melhor_trial_nbayes.number}")
print(f"Parâmetros do melhor trial do Naive Bayes: {melhor_trial_nbayes.params}")

Número do melhor trial da Árvore de Decisão: 6
Parâmetros do melhor trial da Árvore de Decisão: {'profundidade': 91, 'critério': 'gini', 'min_exemplos_split': 110, 'min_exemplos_folha': 76, 'num_max_features': 0.9539556990162101, 'normalizar': True, 'tipo_norm': 'MaxAbs'}
